# Stock Market Question Answering Agent

- answer questions about individual stocks using (mostly) OpenBB functions
- uses the OpenAPI tools functionality
- define a set of functions
- when prompting, give OpenAI the set of functions and their descriptions in a specified format
- if OpenAI needs to run a function to respond to the prompt, it will provide the function signature and ask you to provide the return values before continuing
  

In [1]:
import os
import dotenv
import warnings
from datetime import datetime, timedelta

import yaml

import pandas as pd

import IPython
from IPython.display import HTML, Image, Markdown, display

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.prompts import PromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain.memory import ChatMessageHistory
from langchain.tools import StructuredTool

import openbb
from openbb import obb
from openbb_core.app.model.obbject import OBBject

import openai
from openai import OpenAI
import tiktoken

import langchain
from langchain_core.tools import StructuredTool
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

# langchain is still on pydantic v1
import pydantic.v1
from pydantic.v1 import BaseModel, Field # <-- Uses v1 namespace

import wikipedia

# paid API for edgar filings
import sec_api
from sec_api import QueryApi, ExtractorApi

# free API for edgar filing
from sec_downloader import Downloader
import sec_parser as sp

dotenv.load_dotenv()

from BB_agent_tool import BB_agent_tool, get_response, agent_query, bb_agent_system_prompt

# turn off excessive warnings
warnings.filterwarnings('ignore')


In [2]:
print(f'pandas         {pd.__version__}')
print(f'obb            {obb.system.version}')
print(f'openai         {openai.__version__}')
print(f'langchain      {langchain.__version__}')
print(f'pydantic       {pydantic.v1.__version__}')
print(f'wikipedia      {wikipedia.__version__}')


pandas         2.2.2
obb            4.1.7
openai         1.30.1
langchain      0.2.0
pydantic       1.10.15
wikipedia      (1, 4, 0)


# Connect to OpenBB

In [3]:
# login with email and password
obb.account.login(email=os.environ['OPENBB_USER'], password=os.environ['OPENBB_PW'], remember_me=True)

In [4]:
obb.equity.search("Merck", provider="sec").to_df().head(3)


,symbol,name,cik
0,MRK,"Merck & Co., Inc.",310158


In [5]:
# add all questions from history, use this in streamlit, recheck stuff

symbol = "MRK"
company = "Merck"

In [6]:
obj = obb.equity.price.quote(symbol)
obj


OBBject

id: 0664bb06-60ae-7169-8000-24a6ed243a4d
results: [{'symbol': 'MRK', 'asset_type': None, 'name': 'Merck & Co., Inc.', 'excha...
provider: fmp
warnings: None
chart: None
extra: {'metadata': {'arguments': {'provider_choices': {'provider': 'fmp'}, 'standa...

In [7]:
obj.results[0].model_dump()

{'symbol': 'MRK',
 'asset_type': None,
 'name': 'Merck & Co., Inc.',
 'exchange': 'NYSE',
 'bid': None,
 'bid_size': None,
 'bid_exchange': None,
 'ask': None,
 'ask_size': None,
 'ask_exchange': None,
 'quote_conditions': None,
 'quote_indicators': None,
 'sales_conditions': None,
 'sequence_number': None,
 'market_center': None,
 'participant_timestamp': None,
 'trf_timestamp': None,
 'sip_timestamp': None,
 'last_price': 130.99,
 'last_tick': None,
 'last_size': None,
 'last_timestamp': datetime.datetime(2024, 5, 20, 20, 0, 2, tzinfo=datetime.timezone.utc),
 'open': 130.81,
 'high': 131.74,
 'low': 130.65,
 'close': None,
 'volume': 3166944,
 'exchange_volume': None,
 'prev_close': 131.19,
 'change': -0.2,
 'change_percent': -0.0015240000000000002,
 'year_high': 133.1,
 'year_low': 99.14,
 'price_avg50': 127.1046,
 'price_avg200': 114.8178,
 'avg_volume': 8238545,
 'market_cap': 331772781900.0,
 'shares_outstanding': 2532810000,
 'eps': 0.91,
 'pe': 143.95,
 'earnings_announcement':

# Prompt OpenAI 

In [8]:
MODEL = "gpt-4o"

# MAX_INPUT_TOKENS = 65536
MAX_OUTPUT_TOKENS = 4096    # max in current model
MAX_RETRIES = 3
TEMPERATURE = 0


In [9]:
chat = ChatOpenAI(model=MODEL, temperature=0.0, streaming=True)

prompt_template = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                bb_agent_system_prompt,
            ),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )

chain = prompt_template | chat

memory = ChatMessageHistory(llm=chat, max_token_limit=1000)

memory.add_user_message("What is the airspeed velocity of an unladen swallow")

text_response = ""
display_handle = display(HTML("<p>"), display_id=True)

for chunk in chain.stream({"messages": memory.messages}):
    text_response += chunk.content
    display_handle.update(HTML(f"<p>{text_response}</p>"))
    if hasattr(chunk, 'response_metadata'):
        response_metadata = chunk.response_metadata


# Question Answering Agent
### Create tools

In [10]:
# dict to store tools
tool_dict={}


In [11]:
class SymbolLimitSchema(BaseModel):
    symbol: str
    limit: int

class SymbolSchema(BaseModel):
    symbol: str

class QuerySchema(BaseModel):
    query: str = Field(description="The search string to match to the stock symbol.")
    # limit not supported for provider=sec
    # limit: int = Field(description="The maximum number of values to return")

schema_dict = {
    "SymbolSchema": SymbolSchema,
    "QuerySchema": QuerySchema,
    "SymbolLimitSchema": SymbolLimitSchema,
}

In [12]:
# make a custom tool

# not used for custom tool
yaml_get_10k_item1_from_symbol="""
name: get_10k_item1_from_symbol
description: Given a stock symbol, gets item 1 of the company's latest 10-K annual report filing.
openapi_path: null
parameters:
  symbol:
    description: The symbol to get the 10-K item 1 for
    type: string
example_parameter_values:
- symbol: MSFT
"""
yaml.safe_load(yaml_get_10k_item1_from_symbol)

def fn_get_10k_item1_from_symbol(symbol):
    """
    Get item 1 of the latest 10-K annual report filing for a given symbol.

    Args:
        symbol (str): The symbol of the equity.

    Returns:
        str: The item 1 of the latest 10-K annual report filing, or None if not found.

    """
    item1_text = None
    try:
        # sec needs you to identify yourself for rate limiting
        dl = Downloader(os.getenv("SEC_FIRM"), os.getenv("SEC_USER"))
        html = dl.get_filing_html(ticker=symbol, form="10-K")
        elements: list = sp.Edgar10QParser().parse(html)
        tree = sp.TreeBuilder().build(elements)
        sections = [n for n in tree.nodes if n.text.startswith("Item")]
        item1_node = sections[0]
        item1_text = "\n".join([n.text for n in sections[0].get_descendants()])
    except:
        return None
    # always return a list of dicts
    return [{'item1': item1_text}]

get_10k_item1_from_symbol = StructuredTool.from_function(
    func=fn_get_10k_item1_from_symbol,
    name="get_10k_item1_from_symbol",
    description="Given a stock symbol, gets item 1 of the company's latest 10-K annual report filing.",
    args_schema=SymbolSchema
)

tool_dict["get_10k_item1_from_symbol"]= get_10k_item1_from_symbol
tool_response = get_10k_item1_from_symbol.run("MSFT")
len((tool_response)[0]['item1'])


143226

In [13]:
def obb_function_factory(fn_metadata):
    # get obb method object based on path, e.g. /api/v1/equity/price/quote -> obb.equity.price.quote
    parts = fn_metadata["openapi_path"].removeprefix("/api/v1/").split("/")
    op = obb
    for part in parts:
        op = op.__getattribute__(part)
    singular = fn_metadata.get("singular", 0)
    default_args = fn_metadata.get("default_parameters", {})
    override_args = fn_metadata.get("override_parameters", {})

    # return a closure based on value of op etc.
    def tool_fn(**kwargs):
        """call op and return results without obb metadata"""
        # always use override args
        for k, v in override_args.items():
            kwargs[k] = v
        # use default if not already present
        for k, v in default_args.items():
            if k not in kwargs:
                kwargs[k] = v
        retobj = op(**kwargs)
        retlist = [r.model_dump_json() for r in retobj.results]
        if len(retlist):
            if singular == 1:
                # return first
                return retlist[0]
            elif singular == -1:
                # return last
                return retlist[-1]
        # no retlist or not singular
        return retlist

    return tool_fn


In [14]:
# Load obb tool defs from YAML file
with open('obbtools.yaml', 'r') as file:
    fn_yaml_list = yaml.safe_load(file)

for fn_metadata in fn_yaml_list:

    # instantiate tool and add to tool_dict
    tool_dict[fn_metadata["name"]] = StructuredTool.from_function(
        func=obb_function_factory(fn_metadata),
        name=fn_metadata["name"],
        description=fn_metadata["description"],
        args_schema=schema_dict[fn_metadata["args_schema"]]
    )

    # test tool
    print(fn_metadata["name"])
    for p in fn_metadata['example_parameter_values']:
        print(tool_dict[fn_metadata["name"]].run(tool_input=dict(**p)))
    print()


get_equity_search_symbol
['{"symbol":"AVGO","name":"Broadcom Inc.","cik":"1730168"}']

get_equity_price_quote
['{"symbol":"AAPL","asset_type":"EQUITY","name":"Apple Inc.","exchange":"NMS","bid":191.02,"bid_size":200,"bid_exchange":null,"ask":191.08,"ask_size":100,"ask_exchange":null,"quote_conditions":null,"quote_indicators":null,"sales_conditions":null,"sequence_number":null,"market_center":null,"participant_timestamp":null,"trf_timestamp":null,"sip_timestamp":null,"last_price":191.04,"last_tick":null,"last_size":null,"last_timestamp":null,"open":189.325,"high":191.9199,"low":189.01,"close":null,"volume":44212974.0,"exchange_volume":null,"prev_close":189.87,"change":null,"change_percent":null,"year_high":199.62,"year_low":164.08,"ma_50d":174.111,"ma_200d":180.77705,"volume_average":64446867.0,"volume_average_10d":58961950.0,"currency":"USD"}']

get_company_profile_json
['{"symbol":"NVDA","name":"NVIDIA Corporation","cik":"0001045810","cusip":"67066G104","isin":"US67066G1040","lei":nul

### Create agent

In [15]:
prompt_template = ChatPromptTemplate.from_messages(
    [SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template=bb_agent_system_prompt,)),
     MessagesPlaceholder(variable_name='chat_history', optional=True),
     HumanMessagePromptTemplate(prompt=PromptTemplate(
         input_variables=['input'], template='{input}')),
     MessagesPlaceholder(variable_name='agent_scratchpad')]
)

tool_list = list(tool_dict.values())
agent = create_tool_calling_agent(chat, tool_list, prompt_template)
agent_executor = AgentExecutor(agent=agent, tools=tool_list, verbose=True)


In [16]:

def escape_output(text):
    # escape $ characters to avoid markdown interpreting them as latex
    return text.replace('$', '\$', )


In [17]:
agent_executor = AgentExecutor(agent=agent, tools=tool_list, verbose=True)

response = agent_executor.invoke({"input": "get the latest price quote for Southwest Airlines (symbol LUV)"})

display(Markdown(escape_output(response["output"])))



> Entering new AgentExecutor chain...

Invoking: `get_equity_price_quote` with `{'symbol': 'LUV'}`


['{"symbol":"LUV","asset_type":"EQUITY","name":"Southwest Airlines Co.","exchange":"NYQ","bid":28.02,"bid_size":1100,"bid_exchange":null,"ask":28.21,"ask_size":1000,"ask_exchange":null,"quote_conditions":null,"quote_indicators":null,"sales_conditions":null,"sequence_number":null,"market_center":null,"participant_timestamp":null,"trf_timestamp":null,"sip_timestamp":null,"last_price":28.2,"last_tick":null,"last_size":null,"last_timestamp":null,"open":27.84,"high":28.22,"low":27.655,"close":null,"volume":5706037.0,"exchange_volume":null,"prev_close":27.86,"change":null,"change_percent":null,"year_high":39.53,"year_low":21.91,"ma_50d":28.31,"ma_200d":28.7944,"volume_average":9221870.0,"volume_average_10d":7790820.0,"currency":"USD"}']The latest price quote for Southwest Airlines Co. (symbol: LUV) is as follows:

- **Last Price:** $28.20
- **Bid:** $28.02
- **Ask:** $28.21
- **Open:** $27.

The latest price quote for Southwest Airlines Co. (symbol: LUV) is as follows:

- **Last Price:** \$28.20
- **Bid:** \$28.02
- **Ask:** \$28.21
- **Open:** \$27.84
- **High:** \$28.22
- **Low:** \$27.655
- **Previous Close:** \$27.86
- **Volume:** 5,706,037 shares
- **52-Week High:** \$39.53
- **52-Week Low:** \$21.91
- **50-Day Moving Average:** \$28.31
- **200-Day Moving Average:** \$28.79

All prices are in USD.